<a href="https://colab.research.google.com/github/MaxSMCON/Group_2/blob/main/Group_2_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group 2 Project

## Global food in 

in this project we are going to investigate 

In [ ]:
# https://drive.google.com/file/d/1SbW8ns8H5HDE0FVH5pvZ5GkGUdPV_RgY/view?usp=sharing




In [27]:
import pandas as pd
from pandas_profiling import ProfileReport

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use(style='ggplot')
import seaborn as sns
%matplotlib inline
from IPython.display import display
import requests
import io
from io import StringIO
# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import f1_score, plot_confusion_matrix
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
from sklearn import metrics

# StatsModel
from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# 1MfuleO34g6ZP1saJ1Ej7J7p4SCWYkAmG1MfuleO34g6ZP1saJ1Ej7J7p4SCWYkAmG
#https://drive.google.com/file/d/1SbW8ns8H5HDE0FVH5pvZ5GkGUdPV_RgY/view?usp=sharing

# https://drive.google.com/file/d/1SbW8ns8H5HDE0FVH5pvZ5GkGUdPV_RgY/view?usp=sharing

# Loading the dataset from goolgle drive


In [2]:
!pip install googledrivedownloader #black magic
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id="1SbW8ns8H5HDE0FVH5pvZ5GkGUdPV_RgY",
                                    dest_path="./global_food_prices.csv",
                                    unzip=False)
data = pd.read_csv("global_food_prices.csv")
df = data.copy()
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [30]:
url = 'https://raw.githubusercontent.com/MaxSMCON/Group_2/main/Data%20Storage/GDP.csv'
df_gdp = pd.read_csv(url, index_col=0, encoding= 'latin-1' )
df_gdp.head()

,Subject Descriptor,Units,Scale,Country/Series-specific Notes,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Estimates Start After
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,"Gross domestic product, constant prices",National currency,Billions,Source: National Statistics Office Latest actu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453.484,492.903,496.209,554.91,584.658,662.65,688.247,829.924,899.956,958.266,"1,092.12","1,154.18","1,185.31","1,197.01","1,222.92","1,255.29","1,270.22","1,319.90","1,288.87",NaN,2019.0
Albania,"Gross domestic product, constant prices",National currency,Billions,Source: IMF Staff Estimates. Official national...,252.921,234.711,257.243,281.424,306.47,334.359,297.833,324.13,365.912,391.33,423.784,443.009,467.501,493.283,520.544,551.27,584.254,628.073,649.141,673.204,690.339,700.125,707.14,719.688,735.657,760.042,788.941,821.061,838.908,811.13,854.12,2020.0
Algeria,"Gross domestic product, constant prices",National currency,Billions,Source: National Statistics Office Latest actu...,"3,393.45","3,447.74","3,375.25","3,344.88","3,473.58","3,605.57","3,645.24","3,831.14","3,953.74","4,103.98","4,227.10","4,463.82","4,785.21","4,990.98","5,285.44","5,374.48","5,555.75","5,686.88","5,779.70","5,988.70","6,162.37","6,371.89","6,550.30","6,799.22","7,050.79","7,276.41","7,378.28","7,466.82","7,526.56","7,157.75","7,401.85",2019.0
Andorra,"Gross domestic product, constant prices",National currency,Billions,Source: National Statistics Office. Ministry o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.992,2.153,2.251,2.447,2.646,2.789,2.923,2.968,2.803,2.655,2.602,2.602,2.473,2.385,2.445,2.48,2.572,2.581,2.622,2.675,2.354,2.484,2020.0
Angola,"Gross domestic product, constant prices",National currency,Billions,Source: National Statistics Office. Ministry o...,283.298,315.601,350.22,387.031,427.141,475.013,509.567,533.472,545.109,561.76,585.388,665.385,685.279,760.338,874.608,975.606,"1,112.29","1,236.49","1,247.10","1,307.70","1,353.11","1,468.69","1,541.46","1,615.80","1,631.04","1,588.96","1,586.58","1,554.12","1,545.80","1,462.56","1,452.98",2020.0


In [34]:
url1 = 'https://raw.githubusercontent.com/MaxSMCON/Group_2/main/Data%20Storage/cost%20of%20living%202020.csv'
df_cost = pd.read_csv(url1,index_col=0)
df_cost.head()

,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,McMeal($),Unnamed: 9
Rank 2020,,,,,,,,,
1,Bermuda,147.77,85.70,117.54,146.63,152.52,78.17,0.00,NaN
2,Switzerland,125.69,51.20,89.41,124.93,118.55,114.83,15.36,NaN
3,Norway,96.80,32.61,65.54,89.55,96.81,83.40,12.36,NaN
4,Iceland,90.19,38.54,65.04,80.91,97.17,74.78,13.31,NaN
5,Japan,86.22,26.18,56.98,86.84,48.07,78.88,6.70,NaN


In [ ]:
df = data.copy()
df.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


## Renamin columns

In [ ]:
df = df.rename(columns = {'adm0_name' : 'country', 'mkt_name': 'city',} )
df = df.drop(columns= {'adm1_name'})
df.head()

,adm0_id,country,adm1_id,mkt_id,city,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


## Leaving comodity names with the first words

In [ ]:
df['cm_name'] = df['cm_name'].str.split(" ", 1, expand = True)
df['cm_name'].head()

0    Bread
1    Bread
2    Bread
3    Bread
4    Bread
Name: cm_name, dtype: object

In [ ]:
df.head()

,adm0_id,country,adm1_id,mkt_id,city,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,266,Fayzabad,55,Bread,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,266,Fayzabad,55,Bread,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,266,Fayzabad,55,Bread,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,266,Fayzabad,55,Bread,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,266,Fayzabad,55,Bread,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [ ]:
df.nunique()

adm0_id                   98
country                   98
adm1_id                  894
mkt_id                  3266
city                    3235
cm_id                    636
cm_name                  160
cur_id                     1
cur_name                  84
pt_id                      4
pt_name                    4
um_id                    125
um_name                  125
mp_month                  12
mp_year                   32
mp_price              239811
mp_commoditysource         0
dtype: int64

In [ ]:
df.isnull().sum()

adm0_id                     0
country                     0
adm1_id                     0
mkt_id                      0
city                        0
cm_id                       0
cm_name                     0
cur_id                      0
cur_name                    0
pt_id                       0
pt_name                     0
um_id                       0
um_name                     0
mp_month                    0
mp_year                     0
mp_price                    0
mp_commoditysource    2050638
dtype: int64

In [ ]:
df.groupby(['country'])['mp_price'].mean()

country
Afghanistan    23177.212745
Algeria          164.961907
Angola          1038.963442
Argentina         25.776991
Armenia          818.423680
                   ...     
Venezuela       5471.000317
Viet Nam        7541.105711
Yemen           2884.864064
Zambia             8.187023
Zimbabwe         123.749395
Name: mp_price, Length: 98, dtype: float64

<li> Dollar convertion as of today for better representation